<a href="https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials/blob/main/CompressedCosmos/CompressedCosmos_Meteor_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meteor Rate

In this script we convert the ZHR (Zenithal Hourly Rate) of a meteor stream, to the actual expected rate.

In [1]:
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import astropy.units as u
from datetime import datetime
import pytz

# --- Configuration ---
# 1. Observer's Location (Default: Stuttgart, Germany)
LOCATION = EarthLocation(lat=48.5 * u.deg, lon=9.0 * u.deg, height=200 * u.m)
TIMEZONE = 'Europe/Berlin'

# 2. Observation Date and Time
# Change this to your desired observation time.
OBSERVATION_DATETIME_STR = '2025-08-13 02:00:00' # Format: YYYY-MM-DD HH:MM:SS

# 3. Perseids Shower Parameters
PERSEID_RADIANT = SkyCoord(ra=48.0*u.deg, dec=58.0*u.deg, frame='icrs')
# Population index 'r' describes the brightness distribution of meteors.
# For Perseids, it's typically around 2.2 to 2.6.
POPULATION_INDEX = 2.2

# 4. Sky Condition Parameters
# Limiting Magnitude (LM): The magnitude of the faintest star you can see.
# 6.5 = Perfect dark sky, no moon
# 5.0 = Good dark sky with some light pollution
# 4.0 = Suburban sky / significant moonlight
# 3.0 = Urban sky
LIMITING_MAGNITUDE = 4.0

# Cloud cover factor (0.0 = fully cloudy, 1.0 = perfectly clear)
# You can also consider your "personal FOV" here. Like... you can only see XYZ % of the sky at all
# times
CLOUD_FACTOR = 1.0

# Zenithal Hourly Rate of the Perseids. Needs to be updated frequently!
# For example here: https://globalmeteornetwork.org/flux/
zhr = 50

In [2]:
def calculate_expected_rate(location, time_str, tz):
    """
    Calculates the expected meteor rate based on ZHR, location, and time.
    """
    # Convert the observation time string to an Astropy Time object
    local_time = pytz.timezone(tz).localize(datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S'))
    obs_time = Time(local_time)

    # 2. Calculate the altitude of the radiant
    altaz_frame = AltAz(obstime=obs_time, location=location)
    radiant_altaz = PERSEID_RADIANT.transform_to(altaz_frame)
    radiant_altitude = radiant_altaz.alt.deg

    # If the radiant is below the horizon, we can't see any meteors from it.
    if radiant_altitude < 0:
        radiant_altitude = 0

    print(f"--- Calculating for {local_time.strftime('%Y-%m-%d %H:%M:%S %Z')} ---")
    print(f"Base ZHR for this date: {zhr}")
    print(f"Perseid radiant altitude: {radiant_altitude:.2f} degrees")

    # 3. Apply the correction formula
    # Formula: HR = ZHR * F_c / (sin(hR) * r^(6.5 - lm))
    # HR = Hourly Rate
    # ZHR = Zenithal Hourly Rate
    # F_c = Cloud Factor
    # hR = Radiant Altitude in radians
    # r = Population Index
    # lm = Limiting Magnitude

    hR = np.sin(np.radians(radiant_altitude))
    lm = POPULATION_INDEX**(6.5 - LIMITING_MAGNITUDE)

    # Calculate the final expected hourly rate
    hr = zhr * hR / (CLOUD_FACTOR * lm)

    return hr, radiant_altitude

In [3]:
# --- Run the calculation and print results ---
if __name__ == "__main__":
    expected_rate, altitude = calculate_expected_rate(LOCATION, OBSERVATION_DATETIME_STR, TIMEZONE)

    print("\n--- Prediction Results ---")
    print(f"Observer Location: Stuttgart, Germany")
    print(f"Assumed Limiting Magnitude: {LIMITING_MAGNITUDE}")
    print(f"Cloud Cover: {(1 - CLOUD_FACTOR) * 100:.0f}%")
    print("-" * 26)
    print(f"Predicted number of Perseid meteors per hour: ~{expected_rate:.0f}")

--- Calculating for 2025-08-13 02:00:00 CEST ---
Base ZHR for this date: 50
Perseid radiant altitude: 45.49 degrees

--- Prediction Results ---
Observer Location: Stuttgart, Germany
Assumed Limiting Magnitude: 4.0
Cloud Cover: 0%
--------------------------
Predicted number of Perseid meteors per hour: ~5
